### LangGraph Agent - Customer Support multivoice Agent

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_community.vectorstores import PGVector



In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import PGVector

system = """You are Andrea, a knowledgeable and friendly assistant in a telecommunications company. Your expertise lies in various mobile plans and upgrades. Your role is to help users understand their options and assist them with their queries about mobile plans and upgrades. Always respond in a helpful and professional manner.
Always speak to the user with his name.

Username: {username}

Remember, your goal is to make the user feel supported and informed. Always be courteous and clear in your responses.
"""
prompt_template = PromptTemplate.from_template(system)

In [ ]:
import requests
from langchain_core.tools import tool
from typing import Optional
import time

API_URL = "http://127.0.0.1:8000"
ADMIN_USERNAME = "admin1"
ADMIN_PASSWORD = "admin1password"

categories = ["basic", "normal", "premium"]

def login():
    login_response = requests.post(f"{API_URL}/token/", data={"username": ADMIN_USERNAME, "password": ADMIN_PASSWORD})
    if login_response.status_code != 200:
        print(f"Login failed: {login_response.json().get('detail')}")
        return None, f"Login failed: {login_response.json().get('detail')}"
    access_token = login_response.json().get("access_token")
    headers = {"Authorization": f"Bearer {access_token}"}
    print("Login successful, headers obtained")
    return headers, None

def ask_admin(action: str, username: str, category: Optional[str] = None):
    try:
        headers, error = login()
        if error:
            return None, error

        ask_data = {"action": action, "username": username}
        if category:
            ask_data["category"] = category

        print(f"Requesting admin approval with data: {ask_data}")
        response = requests.post(f"{API_URL}/ask_admin/", json=ask_data, headers=headers)
        if response.status_code != 200:
            print(f"Failed to request admin approval: {response.json().get('detail')}")
            return None, f"Failed to request admin approval: {response.json().get('detail')}"

        print("Admin approval requested")
        return "Admin approval requested", None
    except Exception as e:
        print(f"Failed to execute ask_admin. Error: {repr(e)}")
        return None, f"Failed to execute. Error: {repr(e)}"

def wait_for_admin_approval(action: str, username: str, category: str = None):
    print("Waiting for admin approval...")
    while True:
        response = requests.get(f"{API_URL}/check_confirmation/{username}")
        print("RESPONSE:", response.json())
        if response.status_code == 200:
            result = response.json()
            print(f"Received admin approval response: {result}")
            if result.get("message") == "Admin denied the request":
                return "Admin denied your request"
            elif result.get("message") == "Contract created":
                return result
        time.sleep(2)  # Add a delay before retrying to avoid spamming the server


@tool
def create_contract_tool(username: str, category: str):
    """
    Create a new contract for a user with a specific category.

    Args:
        username (str): Username of the user for whom the contract is being created.
        category (str): Category of the contract. Must be one of "basic", "normal", or "premium".

    Returns:
        str: A string indicating the result of the admin approval process and contract creation.
    """
    print(f"Starting contract creation for user: {username}, category: {category}")

    # Step 0: Check if the user already has a contract
    headers, error = login()
    if error:
        print(f"Error during login: {error}")
        return error

    print(f"Fetching contract details for username: {username}")
    user_contract_response = requests.get(f"{API_URL}/contracts/user/{username}", headers=headers)
    if user_contract_response.status_code == 200:
        user_contract = user_contract_response.json()
        print(f"User contract details: {user_contract}")
        # Check if the user has a valid contract category
        if user_contract.get('category') in categories:
            return f"User already has a contract: {user_contract}"
        else:
            print("No valid contract found for the user.")
    elif user_contract_response.status_code == 404:
        print("No contract found for the user.")
    else:
        print(f"Failed to fetch user contract details: {user_contract_response.json().get('detail')}")
        return f"Failed to fetch user contract details: {user_contract_response.json().get('detail')}"

    # Step 1: Request admin approval
    admin_request, error = ask_admin("create", username, category)
    if error:
        print(f"Error during admin approval request: {error}")
        return error

    # Inform that admin approval is requested
    if admin_request == "Admin approval requested":
        # Wait for admin approval
        approval_result = wait_for_admin_approval("create", username, category)
        if isinstance(approval_result, dict) and "denied" in approval_result.get('message', "").lower():
            print(f"Admin denied the request: {approval_result}")
            return approval_result.get('message')
        elif isinstance(approval_result, dict) and approval_result.get('message') == "Contract created":
            print(f"Admin created the contract: {approval_result}")
            return f"Contract successfully created: ID {approval_result['id']}, Category {approval_result['category']}, Contract Time {approval_result['contract_time']}, User ID {approval_result['user_id']}"

    return "Unexpected flow reached"

tools = [create_contract_tool]


In [ ]:
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage

sys_msg = [SystemMessage(content=prompt_template.format(username="aaa"))]
hu_msg = [HumanMessage(content="I would like to create a premium contract please.")]

chat_history = []

messages = sys_msg + chat_history + hu_msg
model = ChatOpenAI()
model_with_tools = model.bind_tools(tools=tools)

result = model_with_tools.invoke(messages)

In [ ]:
messages.append(result)

In [ ]:
result.tool_calls

In [ ]:
from langchain_core.messages import ToolMessage

for tool_call in result.tool_calls:
    print("Use Tool:", tool_call)
    selected_tool = {"create_contract_tool": create_contract_tool}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [ ]:
messages

In [ ]:
model_with_tools.invoke(messages)